In [ ]:
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf
from google.colab import drive
import os
import json
from datetime import datetime
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from statistics import median


In [ ]:

tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
keras.__version__

'3.8.0'

In [ ]:
drive.mount('/content/drive')
dataPath = "/content/drive/My Drive/Colab Notebooks/Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
lines_path = os.path.join(dataPath, "movie_lines.tsv")
convos_path = os.path.join(dataPath, "movie_conversations.tsv")
diag_path =  os.path.join(dataPath, "movie_lines.tsv")

In [ ]:
dfConvo = pd.read_csv(convos_path, encoding='utf-8-sig', sep="\t")

In [ ]:
dfConvo.head()

,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
0,u0,u2,m0,['L198' 'L199']
1,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
2,u0,u2,m0,['L204' 'L205' 'L206']
3,u0,u2,m0,['L207' 'L208']
4,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']


In [ ]:
dfLines = pd.read_csv(lines_path, encoding='utf-8-sig',header = None)
lines = dfLines[0].str.split('\t')
dialogue_lines = list()
for x in lines:
    dialogue_lines.append(x[4])
dialogue_lines[:10]

['They do not!',
 'They do to!',
 'I hope so.',
 'She okay?',
 "Let's go.",
 'Wow',
 "Okay -- you're gonna need to learn how to lie.",
 'No',
 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'Like my fear of wearing pastels?']

In [ ]:
diag_path
VOCAB_SIZE = 5000
print(VOCAB_SIZE)
EMBEDDING_DIM = 500

5000


In [ ]:
len(dialogue_lines)

293368

In [ ]:
keras_tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}\t\n')
keras_tokenizer.fit_on_texts(dialogue_lines)


In [ ]:
list(keras_tokenizer.word_index.items())[:5]


[('you', 1), ('i', 2), ('the', 3), ('to', 4), ('a', 5)]

In [ ]:
text_sequences = keras_tokenizer.texts_to_sequences(dialogue_lines)[:2000]


In [ ]:
MAX_SEQUENCE_LENGTH = int(median(len(sequence) for sequence in text_sequences))
print(MAX_SEQUENCE_LENGTH)

6


In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D, Masking, Embedding, Layer
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
x_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post',truncating='post', value=0)
x_train = x_train.astype('int16')

In [ ]:
x_train.shape

(2000, 6)

In [ ]:
# x_train_rev = list()
# for x_vector in x_train:
#     x_rev_vector = list()
#     for index in x_vector:
#         char_vector = np.zeros(VOCAB_SIZE)
#         char_vector[index] = 1
#         x_rev_vector.append(char_vector)
#     x_train_rev.append(np.asarray(x_rev_vector))
# x_train_rev = np.asarray(x_train_rev)
x_train_rev = np.copy(x_train)


In [ ]:
x_train_rev.shape

(2000, 6)

In [ ]:
def get_seq2seq_model():
    main_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='main_input')

    embed_1 = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM)(main_input)

    lstm_1 = Bidirectional(LSTM(2064))(embed_1)

    repeat_1 = RepeatVector(MAX_SEQUENCE_LENGTH)(lstm_1)

    lstm_3 = Bidirectional(LSTM(2064, return_sequences=True))(repeat_1)

    softmax_1 = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(lstm_3)

    model = Model(main_input, softmax_1)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
# def get_seq2seq_model():
#     main_input = Input(shape=x_train[0].shape, dtype='float32', name='main_input')
#     print(main_input)

#     embed_1 = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
#                         mask_zero=True, input_length=MAX_SEQUENCE_LENGTH) (main_input)
#     print(embed_1)

#     lstm_1 = Bidirectional(LSTM(64, name='lstm_1'))(embed_1)
#     print(lstm_1)

#     repeat_1 = RepeatVector(MAX_SEQUENCE_LENGTH, name='repeat_1')(lstm_1)
#     print(repeat_1)

#     lstm_3 = Bidirectional(LSTM(64, return_sequences=True, name='lstm_3'))(repeat_1)
#     print(lstm_3)

#     softmax_1 = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(lstm_3)
#     print(softmax_1)

#     model = Model(main_input, softmax_1)
#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])

#     return model
from tensorflow.keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(
    monitor='loss',         # or 'val_loss' if you have validation data
    patience=3,             # stop after 3 epochs of no improvement
    restore_best_weights=True
)

seq2seq_model = get_seq2seq_model()
seq2seq_model.fit(x_train, x_train_rev, batch_size=16, epochs=20, verbose=1 , callbacks=[early_stopping])


predictions = seq2seq_model.predict(x_train)
index2word_map = inv_map = {v: k for k, v in keras_tokenizer.word_index.items()}

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 99s 181ms/step - accuracy: 0.2479 - loss: 5.6844
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 175ms/step - accuracy: 0.3390 - loss: 3.9999
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 176ms/step - accuracy: 0.4025 - loss: 3.3069
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 175ms/step - accuracy: 0.4815 - loss: 2.5512
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 175ms/step - accuracy: 0.5702 - loss: 1.8046
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 176ms/step - accuracy: 0.6950 - loss: 1.1255
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 176ms/step - accuracy: 0.8102 - loss: 0.6769
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 177ms/step - accuracy: 0.8938 - loss: 0.3905
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.9336 - loss: 0.2423
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 177ms/step - accuracy: 0.9652 - loss: 0.1428
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 178ms/step - accuracy: 0.9754 - loss: 0.0993
Epoch 12/20
125/125

In [ ]:

# seq2seq_model.save('seq2seq_model.keras')
# from google.colab import files
# files.download('seq2seq_model.keras')

# model_save_path = "/content/drive/My Drive/Colab Notebooks/Data/seq2seq_model.keras"
# seq2seq_model.save(model_save_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
drive.mount('/content/drive')
model_save_path = "/content/drive/My Drive/Colab Notebooks/Data/"
os.makedirs(model_save_path, exist_ok=True)
seq2seq_model.save(model_save_path + 'seq2seq_model.keras')
seq2seq_model.save('seq2seq_model.keras')
from google.colab import files
files.download('seq2seq_model.keras')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('seq2seq_model_2064.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def sequence_to_str(sequence):
    word_list = list()
    for element in sequence:
        index = np.argmax(element) + 1
        word = index2word_map[index]
        word_list.append(word)

    return word_list

In [83]:

for i in range(len(predictions[5:10])):
        predicted_word_list = sequence_to_str(predictions[i])
        actual_len = len(dialogue_lines[i])

        actual_sentence = "Actual: " + dialogue_lines[i][:len(dialogue_lines[i])-3]

        generated_sentence = ""
        for word in predicted_word_list:
            generated_sentence += word + " "

        sent_dict = dict()
        sent_dict["actual"] = actual_sentence.strip()
        sent_dict["generated"] = generated_sentence.strip()
        print(sent_dict)

{'actual': 'Actual: They do n', 'generated': "him i'm no you you you"}
{'actual': 'Actual: They do', 'generated': "him i'm a you you you"}
{'actual': 'Actual: I hope', 'generated': 'the case out you you you'}
{'actual': 'Actual: She ok', 'generated': 'did little you you you you'}
{'actual': "Actual: Let's", 'generated': 'years her you you you you'}
